**Table of contents**<a id='toc0_'></a>    
- [Geo Queries](#toc1_)    
- [Multimatch Queries](#toc2_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

In [2]:
!pip install elasticsearch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 571.2/571.2 kB 3.7 MB/s eta 0:00:000m eta -:--:--


In [1]:
from datetime import datetime
from elasticsearch import Elasticsearch

In [2]:
def pretty_response(response):
    if len(response["hits"]["hits"]) == 0:
        print("Your search returned no results.")
    else:
        for hit in response["hits"]["hits"]:
            id = hit["_id"]
            gender = hit["_source"]["gender"]
            score = hit["_score"]
            displayName = hit["_source"]["displayName"]
            about = hit["_source"]["about"]
            service = hit["_source"]["service"]["text"]
            avgRating = (
                hit["_source"]["avgRating"]
                if "avgRating" in hit["_source"].keys()
                else 0
            )
            age = hit["_source"]["age"]
            pretty_output = f"\nID: {id}\nProvider: {displayName}\nGender: {gender}\nAbout: {about}\nService: {service}\nRating: {avgRating}\nAge: {age}\nScore: {score}"
            print(pretty_output)

In [3]:
# @6e9qxFPQD?@dYPk, SYjAskK7X@?Pie4

client = Elasticsearch(
    hosts=["https://yomap-elastic-dev.fly.dev"],
    basic_auth=("yomapsearch", "@6e9qxFPQD?@dYPk"),
)

In [4]:
resp = client.search(index="profiles", query={"match_all": {}})

In [5]:
len(resp["hits"]["hits"])

10

In [6]:
resp = client.search(
    index="profiles",
    query={"match": {"displayName": "Laissa"}},
)

for profile in resp["hits"]["hits"]:
    print(profile["_source"])

{'gender': '', 'displayName': 'Laissa', 'about': 'Soy Laissa, una apasionada del transporte y la movilidad. Mi interés por el sector automotriz y mi experiencia como conductora de Uber y motociclista me han brindado una comprensión profunda de las necesidades y desafíos de la industria del transporte. Estoy ansiosa por compartir mis conocimientos y conectar con profesionales de ideas afines en este espacio en constante evolución.', 'photo': 'Files/Users/HHJMAWJTi3SpXUH9ZzBcITXJsjE2/file_1707589877576-628091_file_example_JPG_500kB.jpeg', 'blockedProfiles': [], 'userId': 'HHJMAWJTi3SpXUH9ZzBcITXJsjE2', 'network': [], 'tags': [{'weight': 0, 'id': 'RVqoavsXHaFSepN4ZkNX', 'text': 'transport', 'parentId': '', 'usedBy': 2, 'slug': 'transport', 'parentSlug': ''}, {'weight': 0, 'id': '9pYJdcjbrhiaWODpYN5Q', 'text': 'uber', 'parentId': 'RVqoavsXHaFSepN4ZkNX', 'usedBy': 2, 'slug': 'uber', 'parentSlug': 'transport'}, {'weight': 0, 'id': '5xrlfFLhKrYJk6uWpEU5', 'text': 'moto', 'parentId': 'RVqoavsX

In [9]:
resp = client.search(
    index="profiles",
    query={"match": {"tags.text": "depilacion" + " spa" + " taxi" + " transport"}},
)

for profile in resp["hits"]["hits"]:
    print(profile["_source"]["displayName"])

YoMap LG
manpreet
Laissa
Angela 
Emulator 66980917
YM Soporte stage


In [10]:
resp = client.search(
    index="profiles",
    query={"match": {"tags.text": "costurera"}},
)

for profile in resp["hits"]["hits"]:
    print(profile["_source"]["displayName"])

Costura Ana Mer bis


# <a id='toc1_'></a>[Geo Queries](#toc0_)

In [11]:
resp = client.search(
    index="profiles",
    query={
        "bool": {
            "must": {"match": {"tags.text": "transport " + " spa"}},
            "filter": {
                "geo_distance": {
                    "distance": "10km",
                    "location": {"lat": 8.9842224, "lon": -79.5041315},
                }
            },
        },
    },
)

for profile in resp["hits"]["hits"]:
    print(profile["_source"]["displayName"])

Laissa
Angela 
Emulator 66980917
YM Soporte stage
YoMap LG


In [12]:
resp

ObjectApiResponse({'took': 1, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 5, 'relation': 'eq'}, 'max_score': 2.6533737, 'hits': [{'_index': 'profiles', '_id': 'v34DujEeoB8wx4zysUQr', '_score': 2.6533737, '_source': {'gender': '', 'displayName': 'Laissa', 'about': 'Soy Laissa, una apasionada del transporte y la movilidad. Mi interés por el sector automotriz y mi experiencia como conductora de Uber y motociclista me han brindado una comprensión profunda de las necesidades y desafíos de la industria del transporte. Estoy ansiosa por compartir mis conocimientos y conectar con profesionales de ideas afines en este espacio en constante evolución.', 'photo': 'Files/Users/HHJMAWJTi3SpXUH9ZzBcITXJsjE2/file_1707589877576-628091_file_example_JPG_500kB.jpeg', 'blockedProfiles': [], 'userId': 'HHJMAWJTi3SpXUH9ZzBcITXJsjE2', 'network': [], 'tags': [{'weight': 0, 'id': 'RVqoavsXHaFSepN4ZkNX', 'text': 'transport', 'parentId': '',

# <a id='toc2_'></a>[Multimatch Queries](#toc0_)

In [13]:
resp = client.search(
    index="profiles",
    query={"multi_match": {"query": "assistant", "fields": ["tags.text", "about"]}},
)

for profile in resp["hits"]["hits"]:
    print(profile["_source"]["displayName"])

YoMap Assistant
YM Soporte stage


In [14]:
resp = client.search(
    index="profiles",
    query={
        "bool": {
            "must": {
                "multi_match": {"query": "uber", "fields": ["tags.text", "about"]}
            },
            "filter": {
                "geo_distance": {
                    "distance": "100km",
                    "location": {"lat": 8.9842224, "lon": -79.5041315},
                }
            },
        },
    },
)

# for profile in resp["hits"]["hits"]:
#    print(profile["_source"]["displayName"])

pretty_response(resp)


ID: v34DujEeoB8wx4zysUQr
Provider: Laissa
Gender: 
About: Soy Laissa, una apasionada del transporte y la movilidad. Mi interés por el sector automotriz y mi experiencia como conductora de Uber y motociclista me han brindado una comprensión profunda de las necesidades y desafíos de la industria del transporte. Estoy ansiosa por compartir mis conocimientos y conectar con profesionales de ideas afines en este espacio en constante evolución.
Service: transport
Rating: 5
Age: 
Score: 2.9398274

ID: ZXeLeERP2v9n3HEMCL6c
Provider: YoMap LG
Gender: female
About: Servicio de Taxi 24h
Area Bancaria
Service: transport
Rating: 3.86
Age: 35
Score: 1.9639015


# Fuzzy Queries

In [17]:
resp = client.search(
    index="profiles",
    query={
        "bool": {
            "must": {
                "multi_match": {
                    "query": "Nole",
                    "fields": ["tags.text", "tags.slug", "about", "displayName"],
                    "fuzziness": "AUTO",
                }
            },
            "filter": {
                "geo_distance": {
                    "distance": "10km",
                    "location": {"lat": 8.9842224, "lon": -79.5041315},
                }
            },
        },
    },
)

pretty_response(resp)


ID: oH6kfEwBhELwINiidDwJ
Provider: Noel 2
Gender: male
About: 
Service: food
Rating: 4.5
Age: 44
Score: 2.2960446

ID: FCKoeLnfSy09VFRZuYbn
Provider: Noel Xiaomi
Gender: male
About: Noel Xiaomi, de 44 años, es un técnico experimentado en reparación de dispositivos electrónicos. Mi pasión por la tecnología, particularmente por los smartphones y las computadoras, me ha llevado a dominar habilidades de reparación expertas. Estoy comprometido a brindar soluciones efectivas y de alta calidad para restaurar los dispositivos a sus óptimas condiciones de funcionamiento.
Service: repair
Rating: 4.5
Age: 44
Score: 2.2960446

ID: CoZt1VKl9q7tKi88DciX
Provider: YoMap Assistant
Gender: other
About: Hola soy YoMap el asistente virtual de la aplicación. Estoy aquí para ayudarte a encontrar los proveedores de servicios que resuelvan tus problemas.
Service: misc
Rating: 0
Age: 40
Score: 1.3666141

ID: ceqZenAYRFFNZgXrp2Ts
Provider: Readet Ferche
Gender: female
About: ¡Hola! Soy Readet, una apasionada 